In [1]:
# Problem Parameters

max_rooms_per_course = 3 # A course can be alloted to at most 3 rooms
max_courses_per_student_per_day = 2 # A student can have at most 2 courses in a day
weight_minimize_two_exams_per_day = 10.0 # Weight allocated to minimize no. of 2 exams in a day for a student
weight_minimize_rooms_used =  0  # Weight allocated to minimize no. of rooms used for a course
weight_professor_preferred_slots = 50.0  # Weight allocated to get preffered slots for professors


In [2]:
! pip install tabulate
! pip install matplotlib
! pip install reportlab
! pip install pypdf2

In [3]:
import gurobipy as gp
from gurobipy import GRB
from IPython.display import display, Math, Latex
from openpyxl import Workbook
import import_ipynb
import data_utils as data
import time
import pandas as pd
from collections import defaultdict
from tabulate import tabulate
from reportlab.lib import colors
from reportlab.lib.pagesizes import LETTER
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import PyPDF2
import os 


tt = gp.Model('IIITB Exams Timetable')

185
65
2165
{0: True,
 1: True,
 2: True,
 3: True,
 4: True,
 5: True,
 6: True,
 7: True,
 8: True,
 9: True,
 10: True,
 11: True,
 12: True,
 13: True,
 14: True,
 15: True,
 16: True,
 17: True,
 18: True,
 19: True,
 20: True,
 21: True,
 22: True,
 23: True,
 24: True,
 25: True,
 26: True,
 27: True,
 28: True,
 29: True,
 30: True,
 31: True,
 32: True,
 33: True,
 34: True,
 35: True,
 36: True,
 37: True,
 38: True,
 39: True,
 40: True,
 41: True,
 42: True,
 43: True,
 44: True,
 45: True,
 46: True,
 47: True,
 48: True,
 49: True,
 50: True,
 51: True,
 52: True}
course_group {0: 'NotLab', 1: 'NotLab', 2: 'NotLab', 3: 'NotLab', 4: 'CSE Lab', 5: 'NotLab', 6: 'NotLab', 7: 'NotLab', 8: 'NotLab', 9: 'NotLab', 10: 'NotLab', 11: 'NotLab', 12: 'NotLab', 13: 'NotLab', 14: 'NotLab', 15: 'NotLab', 16: 'NotLab', 17: 'NotLab', 18: 'NotLab', 19: 'NotLab', 20: 'NotLab', 21: 'NotLab', 22: 'NotLab', 23: 'NotLab', 24: 'CSE Lab', 25: 'NotLab', 26: 'NotLab', 27: 'NotLab', 28: 'NotLab', 29:

In [4]:
def exam_slots(course: int):
    return int(data.get_course_duration(course)//15)

exam_spans = {c: data.begin_end_pairs(exam_slots(c)) for c in range(data.n_courses)}

E = tt.addVars(
    [
        (c ,b ,e, d)
        for c in range(data.n_courses)
        for b, e in exam_spans[c]
        for d in range(data.n_days)
    ],
    vtype=GRB.BINARY,
    name="E",
)

C = tt.addVars(
    [
        
        (c,r)
        for c in range(data.n_courses)
        for r in data.group_by_cat[data.course_group[c]]
    ],
    vtype=GRB.INTEGER,
    lb=0,
    ub=250,
    name="C",
)



In [5]:

M = tt.addVars(
    [
        (c, r)
        for c in range(data.n_courses)
        # for r in range(data.n_rooms)
        for r in data.group_by_cat[data.course_group[c]]
    ],
    vtype=GRB.BINARY,
    name = 'M',
)

# D1 = tt.addVars(
#     [
#         (c, r)
#         for c in range(data.n_courses)
#         # for r in range(data.n_rooms)
#         for r in data.group_by_cat[data.course_group[c]]
#     ],
#     vtype=GRB.BINARY,
#     name = 'D1',
# )

# D2 = tt.addVars(
#     [
#         (c, r)
#         for c in range(data.n_courses)
#         # for r in range(data.n_rooms)
#         for r in data.group_by_cat[data.course_group[c]]
#     ],
#     vtype=GRB.BINARY,
#     name = 'D2',
# )

# for c,r in M:
#     room_capacity = data.get_room_capacity(r)
#     tt.addConstr(M[c,r] <=1, name = 'M1_constraint')
#     tt.addConstr(M[c,r] <=C[c,r], name = 'M2_constraint')
#     tt.addConstr(M[c,r] >=D1[c,r], name = 'M3_constraint')
#     tt.addConstr(M[c,r] >=C[c,r]-(room_capacity*(1-D2[c,r])), name = 'M4_constraint')
#     tt.addConstr(D1[c,r]+D2[c,r] == 1, name = 'M5_constraint')


for c in range(data.n_courses):  
    for r in data.group_by_cat[data.course_group[c]]:
        tt.addConstr((M[c, r]==0) >> (C[c, r]==0))
        tt.addConstr((M[c, r]==1) >> (C[c, r]>=min(10, data.get_room_capacity(r), data.get_course_strength(c))))

## Hard Constraints

In [6]:


def covering_sessions(s:int, t:int):
    # Return the list of begin-end pairs for session 's' overlapping with time 't'
    return [(b,e) for b,e in exam_spans[s] if  data.overlap(b,e,t)]

In [7]:
# Constraint 1: Each exam is scheduled exactly once
tt.addConstrs(
    (E.sum(c, '*', '*', '*') == 1 for c in range(data.n_courses)),
    name="one_exam_per_course"
)

tt.update()

In [8]:
# make L_ind(c, b, e, d, r) and L(c, b, e, d, r) = E[c, b, e, d]*C[c, r]

L_ind = tt.addVars(
    [
        (c, b, e, d, r)
        for c in range(data.n_courses)
        for b, e in exam_spans[c]
        for d in range(data.n_days)
        # for r in range(data.n_rooms)
        for r in data.group_by_cat[data.course_group[c]]
    ],
    vtype=GRB.BINARY,
    name="L_ind"
)

L = tt.addVars(
    [
        (c, b, e, d, r)
        for c in range(data.n_courses)
        for b, e in exam_spans[c]
        for d in range(data.n_days)
        # for r in range(data.n_rooms)
        for r in data.group_by_cat[data.course_group[c]]
    ],
    vtype=GRB.INTEGER,
    lb=0,
    ub=250,
    name="L"
)

for c in range(data.n_courses):
    for b, e in exam_spans[c]:
        for d in range(data.n_days):
            # for r in range(data.n_rooms):
            for r in data.group_by_cat[data.course_group[c]]:
                # tt.addConstr((L_ind[c, b, e, d, r]==gp.and_(E[c, b, e, d], M[c, r])))
                # tt.addConstr((L_ind[c, b, e, d, r]==1) >> (L[c, b, e, d, r]==C[c, r]))
                # tt.addConstr((L_ind[c, b, e, d, r]==0) >> (L[c, b, e, d, r]==0))
                tt.addConstr(L[c, b, e, d, r] == E[c, b, e, d]*C[c, r])

tt.update()

In [9]:
# # Constraint 2: Room capacity constraint C[c, r]*E[c, b, e, d] <= room_capacity[r] for all rooms

# tt.addConstrs((gp.quicksum(C[c,r]*E[c,b,e,d]
#                           for c in range(data.n_courses)
#                           for b,e in covering_sessions(c, t)) <= data.get_room_capacity(r)
#               for d in range(data.n_days)
#               for r in range(data.n_rooms)
#               for t in range(data.n_times)),
#               name = "room_capacity_constraint"
#              )
# tt.update()

tt.addConstrs((gp.quicksum(L[c,b,e,d,r] for c in data.courses_in_room[r] for b,e in covering_sessions(c,t)) <= data.get_room_capacity(r)
               for d in range(data.n_days)
               for r in range(data.n_rooms)
               for t in range(data.n_times)),
              name = "room_capacity_constraint"
             )
tt.update()


In [10]:
# Constraint 3: All students for a course should write the exam

tt.addConstrs(
    gp.quicksum(C[c,r] for r in data.group_by_cat[data.course_group[c]]) == data.get_course_strength(c)
    for c in range(data.n_courses)
)

tt.update()


In [11]:
# Constraint 4: Student clash constraint

tt.addConstrs((gp.quicksum(E[c, b, e, d] for c in data.intersection_pairs[i] for b, e in covering_sessions(c, t)) <= 1
               for i in range(data.n_intersection_pairs)
               for d in range(data.n_days)
               for t in range(data.n_times)),
              name='student_clash_constraint'
             )

tt.update()

In [12]:
# Constraint 5: No course can have more than 3 rooms alloted




tt.addConstrs(
    (
        gp.quicksum(M[c, r] for r in data.group_by_cat[data.course_group[c]]) <= max_rooms_per_course
        for c in range(data.n_courses)
    ),
    name="course_room_constraint",
)

tt.update()

In [13]:
# # Constraint 6: Lab exams are conducted in labs

# tt.addConstrs(
#     (
#         gp.quicksum(C[c, r] for r in (data.get_not_rooms_course(c)) )== 0
#         for c in range(data.n_courses)
#     ),
#     name="lab_exam_constraint",
# )

# tt.update()

In [14]:
# Constraint 7: No triplet courses should be scheduled in the day

tt.addConstrs(
    (
        (E.sum(c1, '*', '*', d) + E.sum(c2, '*', '*', d) + E.sum(c3, '*', '*', d) <= max_courses_per_student_per_day)
        for c1, c2, c3 in data.intersection_triplets
        for d in range(data.n_days)
    ),
    name="triplet_constraint",
)


{(5, 20, 28, 0): <gurobi.Constr *Awaiting Model Update*>,
 (5, 20, 28, 1): <gurobi.Constr *Awaiting Model Update*>,
 (5, 20, 28, 2): <gurobi.Constr *Awaiting Model Update*>,
 (5, 20, 28, 3): <gurobi.Constr *Awaiting Model Update*>,
 (5, 20, 28, 4): <gurobi.Constr *Awaiting Model Update*>,
 (5, 20, 28, 5): <gurobi.Constr *Awaiting Model Update*>,
 (18, 19, 21, 0): <gurobi.Constr *Awaiting Model Update*>,
 (18, 19, 21, 1): <gurobi.Constr *Awaiting Model Update*>,
 (18, 19, 21, 2): <gurobi.Constr *Awaiting Model Update*>,
 (18, 19, 21, 3): <gurobi.Constr *Awaiting Model Update*>,
 (18, 19, 21, 4): <gurobi.Constr *Awaiting Model Update*>,
 (18, 19, 21, 5): <gurobi.Constr *Awaiting Model Update*>,
 (16, 22, 25, 0): <gurobi.Constr *Awaiting Model Update*>,
 (16, 22, 25, 1): <gurobi.Constr *Awaiting Model Update*>,
 (16, 22, 25, 2): <gurobi.Constr *Awaiting Model Update*>,
 (16, 22, 25, 3): <gurobi.Constr *Awaiting Model Update*>,
 (16, 22, 25, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1

## Soft Constraints

In [15]:
# # 1. No 2 exams on a day for a student

# # minimize summation courseintersection(c1, c2) * ((E[c1, *, *, d] + E[c2, *, *, d]) > 1) for all c1, c2, d



X = tt.addVars(
    [
        (c1, c2, d)
        for c1, c2 in data.intersection_pairs
        for d in range(data.n_days)
    ],
    vtype = GRB.BINARY, 
    name = 'X'
)

for c1, c2 in data.intersection_pairs:
    for d in range(data.n_days):
        tt.addConstr(E.sum(c1, '*', '*', d) + E.sum(c2, '*', '*', d) - X[c1, c2, d] <= 1 ,
                        name = 'soft_constraint1'
                    )

## Objective Function

In [16]:
# obj is summation of X[c1, c2, d] * weight(c1, c2) for all c1, c2, d
# for c1, c2 in data.intersection_pairs:
#     for d in range(data.n_days):
#         obj1 += X[c1, c2, d] 
obj1 = gp.quicksum(X[c1, c2, d]*(len(data.get_intersection_of_courses(c1, c2))) for c1, c2 in data.intersection_pairs for d in range(data.n_days))
# for c1, c2 in data.intersection_pairs:
#         print(len(data.get_intersection_of_courses(c1, c2)))
#         print(data.get_course_title(c1), data.get_course_title(c2))
#         print(data.course_student[c1], data.course_student[c2])
#         print(data.get_intersection_of_courses(c1, c2))
#         print('----------------------------------')


In [17]:
# Soft Constraint 2: minimize the number of rooms used
obj2 = gp.quicksum(M[c,r] for c in range(data.n_courses) for r in data.group_by_cat[data.course_group[c]])



In [18]:
# obj3 = gp.quicksum(E[c, b, e, d] for c in range(data.n_courses) for b, e, d in data.get_preferred_slots(c))
# for c in range(data.n_courses):
#     for b, e, d in data.get_preferred_slots(c):
#         print("c: ", c, "b: ", b, "e: ", e, "d: ", d)
#         obj3 += E[c, b, e, d]

#add veriable Z[c] for each course c. Z[c] = 1 if the course is scheduled in the preferred slot
Z = tt.addVars(
    [
        (c)
        for c in range(data.n_courses)
    ],
    vtype=GRB.BINARY,
    name="Z",
)

tt.addConstrs(
    (Z[c] == gp.quicksum((E[c, b, e, d] for b, e, d in data.get_preferred_slots(c)))
    for c in range(data.n_courses)), 
    name="soft_constraint3"
)

obj3 = E.sum('*', '*', '*', '*')
obj3 -= gp.quicksum(Z[c] for c in range(data.n_courses))
tt.update()

In [19]:
objective =  obj2*weight_minimize_rooms_used + obj3*weight_professor_preferred_slots + obj1*weight_minimize_two_exams_per_day
tt.setObjective(objective, sense=GRB.MINIMIZE)
tt.update()

In [20]:



def cb(model, where):
    if where == GRB.Callback.MIPNODE:
        # Get model objective
        obj = model.cbGet(GRB.Callback.MIPNODE_OBJBST)

        # Has objective changed?
        if abs(obj - model._cur_obj) > 1e-8:
            # If so, update incumbent and time
            model._cur_obj = obj
            model._time = time.time()

    # Terminate if objective has not improved in 60s
    if time.time() - model._time > 90:
        model.terminate()

# tt._cur_obj = float('inf')
# tt._time = time.time()

# tt.optimize(callback=cb)
# # put time limit of 1200 seconds

# run it for 180 sec and dual gap 5 %

tt.setParam('MIPGap', 0.25)
tt.setParam('TimeLimit', 100)
tt.optimize()

Set parameter MIPGap to value 0.25
Set parameter TimeLimit to value 100
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.1 LTS")

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  100
MIPGap  0.25

Academic license 2615578 - for non-commercial use only - registered to sa___@iiitb.ac.in
Optimize a model with 75842 rows, 276293 columns and 2243546 nonzeros
Model fingerprint: 0xaf6a6ba3
Model has 133248 quadratic constraints
Model has 2496 simple general constraints
  2496 INDICATOR
Variable types: 0 continuous, 276293 integer (141797 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+01, 2e+03]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 2e+02]
  GenCon rhs range [4e+00, 1e+01]
  Gen

In [21]:


def clash_report(X_sol, wb):
    wb.append(["Course 1", "Course 2", "Clash"])  
    for (c1, c2) in data.intersection_pairs:
        clash = 0
        for d in range(data.n_days):
            clash += X_sol[c1, c2, d]
        print(c1, c2, clash)
        wb.append([data.course_id_code[c1], data.course_id_code[c2], "True" if clash>=1 else "False"])
        

def exam_slot_report(Esol, Z_sol, wb):
    wb.append(["Course Code", "Title", "size", "CSE/ECE", "day", "duration", "begin", "end", "preferred_time_slot"])

    for c, b, e, d in Esol:
        wb.append([data.course_id_code[c], data.get_course_title(c), data.get_course_strength(c), data.get_dept(c), data.days[d], data.get_course_duration(c), data.times[b], data.times[e], "True" if Z_sol[c] else "False"])

def seating_report(C_sol, Csol, wb):
    wb.append(["Course Code", "Title", "Room", "Room Type", "size", "Room Number", "Capacity", "Seating"])

    for c, r in Csol:
        wb.append([data.course_id_code[c], data.get_course_title(c), data.get_room_num(r), data.get_course_category(c), data.get_course_strength(c), r, data.get_room_capacity(r), C_sol[c, r]])

def make_report(s):
    C_sol = s['C_sol']
    E_sol = s['E_sol']
    M_sol = s['M_sol']
    X_sol = s['X_sol']
    Z_sol = s['Z_sol']
    Esol = []
    for i in E_sol:
        if E_sol[i] > 0:
            Esol.append(i)
    Csol = []
    for i in C_sol:
        if C_sol[i] > 0:
            Csol.append(i)
    wb = Workbook()
    default_sheet = wb.active
    wb.remove(default_sheet)
    clash_sheet = wb.create_sheet(title='Clash Report')
    exam_slot = wb.create_sheet(title='Exam Slot report')
    seating_sheet = wb.create_sheet(title='Seating Report')

    clash_report(X_sol, clash_sheet)
    exam_slot_report(Esol, Z_sol, exam_slot)
    seating_report(C_sol, Csol, seating_sheet)

    save_path = "./timetable_report.xlsx"
    wb.save(save_path)



In [22]:
# extract E C M X values for all best 3 solution counts and print them

def extract_values(model):
    E_sol = model.getAttr('x', E)
    C_sol = model.getAttr('x', C)
    M_sol = model.getAttr('x', M)
    X_sol = model.getAttr('x', X)
    Z_sol = model.getAttr('x', Z)
    return E_sol, C_sol, M_sol, X_sol, Z_sol
# make a 3 pd dataframe for the sol

# E_sol C_sol M_sol X_sol are sets, i want to crerate a dataframe with columns E_sol, C_sol, M_sol, X_sol
def get_bestmax3_solutions(model):
    for i in range(min(1, model.SolCount)):
        s = {}
        model.setParam('SolutionNumber', i)
        E_sol, C_sol, M_sol, X_sol, Z_sol = extract_values(model)
        s['E_sol'] = E_sol
        s['C_sol'] = C_sol
        s['M_sol'] = M_sol
        s['X_sol'] = X_sol
        s['Z_sol'] = Z_sol
        make_report(s)

get_bestmax3_solutions(tt)


16 20 0.0
7 17 0.0
33 36 0.0
10 34 0.0
41 49 0.0
16 29 0.0
44 45 0.0
1 40 0.0
50 52 0.0
48 36 0.0
20 29 0.0
46 48 0.0
5 19 0.0
39 51 0.0
12 34 0.0
11 14 0.0
5 28 1.0
4 30 0.0
40 50 0.0
17 30 0.0
5 37 0.0
9 35 0.0
24 26 0.0
6 20 0.0
16 22 1.0
18 19 0.0
10 36 0.0
41 51 0.0
42 50 0.0
44 47 0.0
1 42 0.0
0 44 0.0
10 45 0.0
20 22 0.0
5 12 0.0
22 28 0.0
2 4 0.0
17 23 0.0
4 23 1.0
12 36 0.0
40 52 0.0
5 30 0.0
12 45 0.0
15 16 0.0
13 28 0.0
25 27 0.0
18 21 0.0
13 37 0.0
42 52 0.0
10 47 0.0
1 44 0.0
11 46 0.0
12 20 0.0
3 17 0.0
31 33 0.0
23 29 0.0
40 45 1.0
5 23 0.0
4 25 0.0
12 47 0.0
27 29 0.0
27 38 0.0
15 18 0.0
16 17 0.0
10 31 0.0
2 27 0.0
42 45 0.0
25 29 0.0
7 23 0.0
18 23 0.0
46 36 0.0
11 48 0.0
20 26 0.0
22 23 0.0
5 7 0.0
29 38 0.0
5 16 0.0
4 18 0.0
31 35 0.0
9 14 0.0
40 47 0.0
3 28 0.0
4 27 0.0
12 31 0.0
19 27 0.0
2 20 0.0
16 19 0.0
7 16 1.0
33 35 0.0
10 33 0.0
2 29 0.0
42 47 0.0
11 32 0.0
16 28 0.0
6 29 0.0
20 28 0.0
4 11 0.0
21 27 0.0
39 41 0.0
3 21 0.0
5 18 0.0
4 20 0.0
22 25 0.0
12 33 

In [23]:
E_sol, C_sol, M_sol, X_sol, Z_sol = extract_values(tt)

solution = []
for i in E_sol:
    if E_sol[i] > 0:
        solution.append(i)

In [24]:
solution_rooms=[]
for i in C_sol:
    if C_sol[i] > 0:
        solution_rooms.append(i)

In [25]:
schedule_by_day = defaultdict(list)

for c, b, e, d in solution:
    rooms = []
    for r in data.group_by_cat[data.course_group[c]]:
        if C_sol[c, r] > 0:
            rooms.append(f"{data.get_room_num(r)} (Capacity: {C_sol[c, r]})")
    entry = {
        "Course": data.course_id_code[c],
        "Strength": data.get_course_strength(c),
        "Time": f"{data.times[b]} - {data.times[e]}",
        "Rooms": ", ".join(rooms),
        "start": data.times[b]
    }
    day = data.days[d]
    schedule_by_day[day].append(entry)

weekdays = ["Mon", "Tue", "Wed", "Thu", "Fri","Sat"]

print("EXAM TIMETABLE")
print("=" * 50)
for day in weekdays:
    if day in schedule_by_day:
        sorted_entries = sorted(
            schedule_by_day[day],
            key=lambda entry: (entry["start"], entry["Strength"])
        )
        for entry in sorted_entries:
            del entry["start"]
        print(f"\n{day}")
        print("-" * 50)
        print(tabulate(sorted_entries, headers="keys", tablefmt="grid"))

EXAM TIMETABLE

Mon
--------------------------------------------------
+------------+------------+---------------+-------------------------------------------------------------------------------------------------+
| Course     |   Strength | Time          | Rooms                                                                                           |
+============+============+===============+=================================================================================================+
| DHS 314    |         25 | 09:30 - 11:30 | A304/A305 (Capacity: 25.0)                                                                      |
+------------+------------+---------------+-------------------------------------------------------------------------------------------------+
| DAS 101P-A |         96 | 09:30 - 12:00 | R107/R108 (LAB) (Capacity: 96.0)                                                                |
+------------+------------+---------------+----------------------------------